In [2]:
import pandas as pd

In [3]:
cic_columns = [
    'Flow ID', ' Source IP', ' Source Port', ' Destination IP',
    ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
    ' Total Fwd Packets', ' Total Backward Packets',
    'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max', ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
    'Bwd Packet Length Max', ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
    ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
    ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
    ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
    ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
    ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
    ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
    ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
    ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
    'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count',
    ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
    ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio',
    ' Average Packet Size', ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk',
    ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
    ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets',
    ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes',
    'Init_Win_bytes_forward', ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean',
    ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std',
    ' Idle Max', ' Idle Min', ' Label'
] 

In [4]:
import json

file_path = "results/suricata_eve_all.json"
with open(file_path, "r") as f:
    eve_data = json.load(f)

In [5]:
eve_data

[{'timestamp': '2025-07-10T16:49:25.691024+0000',
  'flow_id': 1523555128178732,
  'in_iface': 'eth0',
  'event_type': 'ssh',
  'src_ip': '172.24.0.6',
  'src_port': 34674,
  'dest_ip': '172.24.0.4',
  'dest_port': 22,
  'proto': 'TCP',
  'pkt_src': 'wire/pcap',
  'tx_id': 0,
  'ssh': {'client': {'proto_version': '2.0',
    'software_version': 'OpenSSH_9.2p1'},
   'server': {'proto_version': '2.0', 'software_version': 'OpenSSH_8.9p1'}}},
 {'timestamp': '2025-07-10T16:49:29.861424+0000',
  'event_type': 'stats',
  'stats': {'uptime': 28,
   'capture': {'kernel_packets': 42,
    'kernel_drops': 0,
    'errors': 0,
    'afpacket': {'busy_loop_avg': 0,
     'polls': 836,
     'poll_signal': 0,
     'poll_timeout': 810,
     'poll_data': 26,
     'poll_errors': 0,
     'send_errors': 0}},
   'decoder': {'pkts': 42,
    'bytes': 9950,
    'invalid': 0,
    'ipv4': 42,
    'ipv6': 0,
    'ethernet': 42,
    'arp': 0,
    'unknown_ethertype': 0,
    'chdlc': 0,
    'raw': 0,
    'null': 0,
   

In [6]:
suricata_keys = set()
for entry in eve_data:
    suricata_keys.update(entry.keys())
    for subfield in ('flow', 'tcp', 'stats', 'alert'):
        if subfield in entry and isinstance(entry[subfield], dict):
            suricata_keys.update(f"{subfield}.{k}" for k in entry[subfield].keys())

In [7]:
#suricata_keys

In [ ]:
cic_to_suricata = {
    'ACK Flag Count': 'tcp.ack',
    'Destination IP': 'dest_ip',
    'Destination Port': 'dest_port',
    'FIN Flag Count': 'tcp.fin',
    'Flow Duration': 'flow.start + flow.end',
    'Flow ID': 'flow_id',
    # 'Label': 'alert.signature',
    'PSH Flag Count': 'tcp.psh',
    'Protocol': 'proto',
    'SYN Flag Count': 'tcp.syn',
    'Source IP': 'src_ip',
    'Source Port': 'src_port',
    # 'Timestamp': 'timestamp',
    'Total Backward Packets': 'flow.pkts_toclient',
    'Total Fwd Packets': 'flow.pkts_toserver',
    'Total Length of Bwd Packets': 'flow.bytes_toclient',
    'Total Length of Fwd Packets': 'flow.bytes_toserver'
}

In [15]:
cic_to_suricata.keys()

dict_keys(['ACK Flag Count', 'Destination IP', 'Destination Port', 'FIN Flag Count', 'Flow Duration', 'Flow ID', 'PSH Flag Count', 'Protocol', 'SYN Flag Count', 'Source IP', 'Source Port', 'Timestamp', 'Total Backward Packets', 'Total Fwd Packets', 'Total Length of Bwd Packets', 'Total Length of Fwd Packets'])

In [10]:
results = []
for col in cic_columns:
    col_stripped = col.strip()
    mapped = cic_to_suricata.get(col_stripped, 'UNKNOWN')
    if '+' in mapped:
        parts = [p.strip() for p in mapped.split('+')]
        exists = all(part in suricata_keys for part in parts)
    else:
        exists = mapped in suricata_keys
    results.append({
        'CIC Column': col_stripped,
        'Mapped Suricata Field': mapped,
        'Mapped?': mapped != 'UNKNOWN',
        'Exists in Suricata Keys': exists
    })
df = pd.DataFrame(results)

In [11]:
df.to_json('results/mapped_suricata_fields.json', orient='records', indent=4)

In [12]:
len(df.loc[df['Exists in Suricata Keys']])

17

In [13]:
df

,CIC Column,Mapped Suricata Field,Mapped?,Exists in Suricata Keys
0,Flow ID,flow_id,True,True
1,Source IP,src_ip,True,True
2,Source Port,src_port,True,True
3,Destination IP,dest_ip,True,True
4,Destination Port,dest_port,True,True
...,...,...,...,...
80,Idle Mean,UNKNOWN,False,False
81,Idle Std,UNKNOWN,False,False
82,Idle Max,UNKNOWN,False,False
83,Idle Min,UNKNOWN,False,False
